In [1]:
import datetime
import numpy as np
import alpaca_trade_api as  tradeapi
import time
import talib as ta
import sys
sys.path.insert(0, '../')
import pandas as pd
from datetime import timedelta

In [2]:
import dtale
import plotly
import dash

In [3]:
from config import key_id, secret_key, poly_key_id, alpha_api_key 
import cross_signal

In [4]:
import requests

In [5]:
#make sure to import these packages
from pandas import DataFrame
import talib.abstract as tab
#abstract automatically extracts the columns and periods from the data itself, no input required



#feel free to add or subtract more indicators for any other strategy you want to use
def populateindicators(dataframe) -> DataFrame:
        #exponential moving averages
        dataframe['ema_high'] = tab.EMA(dataframe, timeperiod=4, price='high')
        dataframe['ema_close'] = tab.EMA(dataframe, timeperiod=4, price='close')
        dataframe['ema_low'] = tab.EMA(dataframe, timeperiod=4, price='low')
        
        dataframe['dema_close'] = tab.DEMA(dataframe, timeperiod=4, price='close')
        dataframe['sema_high'] = tab.EMA(dataframe, timeperiod=10, price='high')
        dataframe['sema_close'] = tab.EMA(dataframe, timeperiod=10, price='close')
        dataframe['sema_low'] = tab.EMA(dataframe, timeperiod=10, price='low')
        
        dataframe['ssema_close'] = tab.EMA(dataframe, timeperiod=35, price='close')
        dataframe['sssema_close'] = tab.EMA(dataframe, timeperiod=40, price='close')
        dataframe['ssssema_close'] = tab.EMA(dataframe, timeperiod=45, price='close')
        dataframe['chaikosc'] = ta.ADOSC(dataframe['high'],dataframe['low'],dataframe['close'], np.asarray(dataframe['volume'], dtype='float'))
        
        dataframe['mfi'] = ta.MFI(dataframe['high'],dataframe['low'],dataframe['close'], np.asarray(dataframe['volume'], dtype='float'), timeperiod=14)
        dataframe['ultosc'] = ta.ULTOSC(dataframe['high'],dataframe['low'],dataframe['close'])
        dataframe['obv'] = ta.OBV(dataframe['close'], np.asarray(dataframe['volume'], dtype='float'))
        
        
        dataframe['minusdi'] = tab.MINUS_DI(dataframe, timeperiod=14)
        dataframe['longminusdi'] = tab.MINUS_DI(dataframe, timeperiod=28)
        dataframe['plusdi'] = tab.PLUS_DI(dataframe, timeperiod=14)
        
        dataframe['longminusdi'] = tab.MINUS_DI(dataframe, timeperiod=35)
        dataframe['longplusdi'] = tab.PLUS_DI(dataframe, timeperiod=35)
        dataframe['splusdi'] = tab.PLUS_DI(dataframe, timeperiod=7)
        dataframe['sma']=tab.SMA(dataframe, timeperiod=10, price='close')
        dataframe['vwap']=dataframe['sma']*dataframe['volume']
        
        dataframe['emaminussdi'] = ta.MINUS_DI(dataframe['ema_high'],dataframe['ema_low'],dataframe['ema_close'])
        dataframe['emaplusdi'] = ta.PLUS_DI(dataframe['ema_high'],dataframe['ema_low'],dataframe['ema_close'])
        dataframe['emaadx'] = ta.ADX(dataframe['ema_high'],dataframe['ema_low'],dataframe['ema_close'])
        
        dataframe['upperband'], dataframe['midband'], dataframe['lowerband'] = ta.BBANDS(dataframe['close'], timeperiod=5, nbdevup=1, nbdevdn=1, matype=0)
        
        dataframe['macd'],dataframe['macdsig'],dataframe['hist']= ta.MACD(dataframe['close'], fastperiod=12, slowperiod=26, signalperiod=9)
        dataframe['rsi'] = tab.RSI(dataframe, timeperiod=14, price='close')
        dataframe['adx'] = tab.ADX(dataframe)
        dataframe['shortadx'] = tab.ADX(dataframe, timeperiod=10)
        dataframe['srsi'] = tab.RSI(dataframe, timeperiod=3, price='close')
        dataframe["upperband"], dataframe["middleband"], dataframe["lowerband"] = ta.BBANDS(dataframe['close'],  nbdevup=1.5, nbdevdn=1.5)
        
        dataframe['ema_ultosc'] = ta.ULTOSC(dataframe['ema_high'],dataframe['ema_low'],dataframe['ema_close'])
        dataframe['ema_rsi'] = ta.RSI(dataframe['ema_close'], timeperiod=3)
        dataframe['fema_rsi'] = ta.RSI(dataframe['ema_close'], timeperiod=2)
        dataframe['sema_rsi'] = ta.RSI(dataframe['ema_close'], timeperiod=6)
        
        
        dataframe['beta'] = tab.BETA(dataframe)
        
        dataframe['macd1'],dataframe['macdsig1'],dataframe['hist1']= ta.MACD(dataframe['close'], fastperiod=8, slowperiod=10, signalperiod=7)
        
        
        """dataframe['macd_cross1'] =crossed_above(dataframe['macd1'],dataframe['macdsig1'])
        dataframe['macd_cross'] =crossed_above(dataframe['macd'],dataframe['macdsig'])
        dataframe['adxcross1'] =crossed_above(dataframe['adx'],17)
        dataframe['adxcross2'] =crossed_above(dataframe['adx'],27)
        dataframe['adxcross3'] =crossed_above(dataframe['adx'],35)
        """
        # required for graphing

        return dataframe

In [6]:
#alpha vantage
#  standard API volume limit (5 API requests per minute; 500 API requests per day)
def get_intraday_data(symbol, interval,alpha_api_key):
    api_key = alpha_api_key
    api_url = f'https://www.alphavantage.co/query?function=TIME_SERIES_INTRADAY&symbol={symbol}&interval={interval}&apikey={api_key}'
    raw_df = requests.get(api_url).json()
    df = pd.DataFrame(raw_df[f'Time Series ({interval})']).T
    df = df.rename(columns = {'1. open': 'open', '2. high': 'high', '3. low': 'low', '4. close': 'close', '5. volume': 'volume'})
    for i in df.columns:
        df[i] = df[i].astype(float)
    df.index = pd.to_datetime(df.index)
    df = df.iloc[::-1]
    return df


In [7]:
#The url at the end is what makes a paper trading bot
api = tradeapi.REST(key_id, secret_key)
api2=tradeapi.REST(poly_key_id, secret_key)

In [8]:
df=get_intraday_data('SPY','5min',alpha_api_key)
#This is to make the index available for dtale graphs
df.index.name = 'date_index'
df

,open,high,low,close,volume
date_index,,,,,
2021-05-13 11:45:00,410.58,410.58,409.3400,409.360,1328995.0
2021-05-13 11:50:00,409.35,410.01,409.2101,409.970,1162479.0
2021-05-13 11:55:00,409.96,410.21,409.7800,410.065,598494.0
2021-05-13 12:00:00,410.05,410.22,409.4000,409.665,1073364.0
2021-05-13 12:05:00,409.67,410.39,409.4700,409.800,1251471.0
...,...,...,...,...,...
2021-05-13 19:40:00,410.97,411.00,410.2200,410.460,17523.0
2021-05-13 19:45:00,410.46,410.92,410.4600,410.920,6846.0
2021-05-13 19:50:00,410.99,411.17,410.9100,410.910,7672.0


In [9]:
data = populateindicators(df)
d = dtale.show(df)

In [10]:
d

In [11]:
d.notebook_charts()